In [1]:
import sys  
sys.path.insert(0, '../')
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Pad
import os
import pickle
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets

from tqdm import tqdm
import seaborn as sns
from pathlib import Path

import preprocessing as pre
import visualize as vis
from models import LightningDiffusion as ld


%load_ext autoreload
%autoreload 2


/data/cb/mihirb14/mambaforge/envs/daifuku/lib/python3.9/site-packages/scipy-1.7.3-py3.9-linux-x86_64.egg/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_path = "/data/cb/ruochiz/scHiC/"
data_names = ["m3c_mouse_brain_small",  "m3c_Tian_et_al", "m3C_hg38_M1C_old",  "m3c_human_pfc_old", "m3c_Heffel_et_al"]
selection = int(input("Pick dataset (1. Small Mouse Brain | 2. Tian et al): "))-1
data_name = data_names[selection]
dataset_path = os.path.join(data_path,data_name)

if selection ==0:
    config = pre.get_config("/data/cb/mihirb14/projects/Daifuku/configs/config_m3c_mouse_brain_small_1M.json")
    train_config = pre.get_config("/data/cb/mihirb14/projects/Daifuku/configs/train_configs/config_joint_smallmouse.json")
elif selection == 1:
    config = pre.get_config("/data/cb/mihirb14/projects/Daifuku/configs/config_m3c_Tian_HumanBrain_1M.JSON")
    train_config = pre.get_config("/data/cb/mihirb14/projects/Daifuku/configs/train_configs/config_joint_tianhumanbrain.json")

train_config["normalization"] = "none" # for visualization purposes
config["train_config"] = train_config

dataset_path
# dataset_hic_path = os.path.join(dataset_path,"contact_pairs_filter")

'/data/cb/ruochiz/scHiC/m3c_Tian_et_al'

### Joint Dataloading (Paired dataloader for pseudobulk and single cell)

In [4]:
sc_indices_file = "/data/cb/mihirb14/projects/Daifuku/data/m3c_Tian_et_al/contactmaps_1M/aggregated/sc/chr10_sparse_adj.npy"
bulk_file = "/data/cb/mihirb14/projects/Daifuku/data/m3c_Tian_et_al/contactmaps_1M/aggregated/bulk/chr10.npy"

sc_indices = np.load(sc_indices_file)
bulk_sparse_maps = np.load(bulk_file)


# cell_numbers = range(1,143761)

# ogpath = 
# sparse = np.load("/data/cb/mihirb14/projects/Daifuku/data/m3c_Tian_et_al/contactmaps_1M/raw/chr10_sparse_adj.npy",allow_pickle=True)
# all_maps = [map.todense() for map in sparse]

    
# save = np.stack(all_maps,axis=0)
# np.save(os.path.join(ogpath,f"aggregated/chr10.npy"),save)
# save.shape

Takes ~4 minutes to load in Tian Human brain dataset

In [4]:
joint_hic_dataset = pre.JointHiCDataset(config=config)
joint_hic_dataloader = DataLoader(joint_hic_dataset, batch_size=1) # batch size 1 just for testing purposes

Bulk Sparse Dataset Size 1150080
sc Sparse Dataset Size 1150080


In [5]:
batch = next(iter(joint_hic_dataloader))
bulk,sc = batch.squeeze()[0],batch.squeeze()[1]
print(bulk.shape, sc.shape)
visualized = [vis.visualize_contact_map(bulk,zmax=1, title="Bulk Contact Map 1"),vis.visualize_contact_map(sc,zmax=1, title="Single Cell Contact Map 1") ]
display(vis.visualize_multiple_jupyter(visualized))

torch.Size([136, 136]) torch.Size([136, 136])


    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<…

### Load in Daifuku Model

In [10]:
daifuku = ld.LightningDiffusion(train_config=train_config)
type(daifuku)

Conditioning has been set to true. Using conditional Unet as score model.


models.LightningDiffusion.LightningDiffusion

In [16]:
daifuku(bulk.unsqueeze(0).unsqueeze(0),condition=sc.unsqueeze(0).unsqueeze(0))

(tensor([[[[ 2.5673,  3.8142,  4.1953,  ...,  0.0104, -0.3448, -0.6133],
           [-0.0446,  0.1391,  1.0921,  ...,  2.8347,  0.2233, -0.9722],
           [-3.4471, -5.9073, -2.4648,  ...,  3.5274,  2.8734,  4.4010],
           ...,
           [-2.7203, -4.7291, -7.8794,  ..., -4.6024,  3.0866,  8.3258],
           [-2.9051, -3.8921, -6.5966,  ..., -6.8129,  1.6119,  8.3386],
           [-0.3797,  1.4915, -2.4056,  ..., -8.2699, -1.3209,  2.5943]]]],
        grad_fn=<ConvolutionBackward0>),
 tensor(9.7999, grad_fn=<MeanBackward0>))

In [17]:
daifuku.training_step(next(iter(joint_hic_dataloader)),0)

torch.Size([1, 1, 136, 136]) torch.Size([1, 1, 136, 136])


AttributeError: 'NoneType' object has no attribute 'experiment'